<a href="https://colab.research.google.com/github/GermanM3/GermanM3/blob/master/Capital_Efficiency_Valuation_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pykrx finance-datareader ta

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

from pykrx import stock
import FinanceDataReader as fdr

In [ ]:
# === 1) 파라미터 ===
TICKER = "044450"
NAME = "KSS해운"
YEARS = 10
TARGET_PBR = 1.0

In [29]:
# === 2) 날짜 구간 설정 ===
today = datetime.today()
end_str = today.strftime("%Y%m%d")

# === 3) 현재가, 시총, 주식수 ===
px = stock.get_market_ohlcv_by_date(fromdate=end_str, todate=end_str, ticker=TICKER)
if px.empty:
    yday = (today - timedelta(days=7)).strftime("%Y%m%d")
    px = stock.get_market_ohlcv_by_date(fromdate=yday, todate=end_str, ticker=TICKER).tail(1)
close = float(px['종가'].iloc[-1])

# --- 4) 펀더멘털 시계열 최적화 (현재 연도 데이터 포함) ---
annual_list = []
# 지난 10개 연도와 현재 연도를 포함한 리스트 생성
years_to_fetch = list(range(today.year - YEARS, today.year + 1))

for year in years_to_fetch:
    end_date_str = f'{year}1231'

    # 현재 연도는 오늘 날짜까지의 데이터를 가져오도록 설정
    if year == today.year:
        end_date_str = today.strftime("%Y%m%d")
        # 데이터가 없을 경우를 대비해 넉넉하게 15일치 조회
        start_date = (today - timedelta(days=15)).strftime("%Y%m%d")
    else:
        # 과거 연도는 연말 데이터를 가져오도록 설정
        start_date = (datetime.strptime(end_date_str, "%Y%m%d") - timedelta(days=15)).strftime("%Y%m%d")

    try:
        fund_data = stock.get_market_fundamental_by_date(start_date, end_date_str, TICKER)
        if not fund_data.empty:
            annual_list.append(fund_data.tail(1))
    except Exception as e:
        print(f"Error fetching data for {year}: {e}")
        continue

annual = pd.concat(annual_list, axis=0).sort_index()

annual = annual[annual['BPS'] > 0]
annual['ROE_%'] = (annual['EPS'] / annual['BPS']) * 100

# 10년 평균 ROE (현재 연도 데이터 포함)
avg_roe = float(annual['ROE_%'].mean())
cur_bps = float(annual['BPS'].iloc[-1])

# === 5) 채권형 주식 방식 계산 ===
future_bps = cur_bps * (1 + avg_roe/100) ** YEARS
price_target = future_bps * TARGET_PBR
irr = (price_target / close) ** (1/YEARS) - 1

# 보조: 현재 PBR, EPS, ROE 상세
cur_row = annual.iloc[-1]
cur_pbr = float(cur_row['PBR']) if 'PBR' in cur_row else np.nan
cur_eps = float(cur_row['EPS']) if 'EPS' in cur_row else np.nan

print("="*30)
print(f"종목: {NAME} ({TICKER})")
print("="*30)
print(f"✓ 현재가: {close:,.0f}원")
print(f"✓ 현재 PBR: {cur_pbr:.2f}배")
print(f"✓ 최근 ROE: {cur_row['ROE_%']:.2f}%")
print(f"✓ 10년 평균 ROE: {avg_roe:.2f}%")
print(f"✓ 현재 BPS: {cur_bps:,.0f}원")
print(f"✓ 10년 후 예상 BPS: {future_bps:,.0f}원")
print(f"✓ 10년 후 목표 주가: {price_target:,.0f}원")
print(f"✓ 연평균 기대수익률 (IRR): {(irr*100):.2f}%")
print("="*30)

종목: KSS해운 (044450)
✓ 현재가: 9,300원
✓ 현재 PBR: 0.39배
✓ 최근 ROE: 10.84%
✓ 10년 평균 ROE: 9.48%
✓ 현재 BPS: 23,452원
✓ 10년 후 예상 BPS: 58,013원
✓ 10년 후 목표 주가: 58,013원
✓ 연평균 기대수익률 (IRR): 20.09%


In [30]:
# === 7) 부가: 시각화용 테이블 ===
view = annual[['EPS','BPS','PBR']].copy()
view['ROE_%(EPS/BPS)'] = (view['EPS']/view['BPS']*100).round(2)
print("\n[연도별 EPS/BPS/PBR 및 근사 ROE(%)]")
print(view)


[연도별 EPS/BPS/PBR 및 근사 ROE(%)]
             EPS    BPS   PBR  ROE_%(EPS/BPS)
날짜                                           
2015-12-30  1815  15902  1.08           11.41
2016-12-29  1016   9154  0.88           11.10
2017-12-28   585  10177  0.89            5.75
2018-12-28  1120   9806  0.66           11.42
2019-12-30   914  10756  0.72            8.50
2020-12-30  1034  12026  0.86            8.60
2021-12-30   584  11592  1.02            5.04
2022-12-29  2497  15261  0.61           16.36
2023-12-28  2016  17995  0.51           11.20
2024-12-30   755  18624  0.47            4.05
2025-08-20  2543  23452  0.39           10.84
